## Visión por Computadora 1 - Cohorte 17
## Trabajo Práctico 3
## Paola Cartala - Florentino Arias

## Parte 1: Detección del logo en cada imagen sin falsos positivos.
Vamos a utilizar cv2.matchTemplate() para implementar el método de template matching. Filtramos los resultados para evitar falsos positivos.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Cargar el template
template = cv2.imread('template.png', cv2.IMREAD_GRAYSCALE)
template_h, template_w = template.shape

# Lista de imágenes a procesar
image_paths = ['images/image1.png', 'images/image2.png', 'images/image3.png']

for path in image_paths:
    # Leer la imagen
    image = cv2.imread(path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Aplicar template matching
    result = cv2.matchTemplate(gray_image, template, cv2.TM_CCOEFF_NORMED) # Utilizamos correlación normalizada (TM_CCOEFF_NORMED) para medir la similitud.
    threshold = 0.8  # Ajustar el umbral para evitar falsos positivos
    locations = np.where(result >= threshold)
    
    # Dibujar los bounding boxes para cada coincidencia
    for pt in zip(*locations[::-1]):
        cv2.rectangle(image, pt, (pt[0] + template_w, pt[1] + template_h), (0, 255, 0), 2)
        confidence = result[pt[1], pt[0]]
        cv2.putText(image, f'{confidence:.2f}', (pt[0], pt[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    
    # Mostrar el resultado
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(f'Detección en {path}')
    plt.axis('off')
    plt.show()


## Parte 2: Algoritmo para múltiples detecciones en 'coca_multi.png'
Para múltiples instancias del logotipo en la imagen coca_multi.png tenemos que ajustar la lógica para permitir múltiples detecciones.

Se dibujan todos los bounding boxes que cumplen con el umbral definido (threshold).

In [ ]:
# Cargar la imagen coca_multi.png
multi_image_path = 'images/coca_multi.png'
multi_image = cv2.imread(multi_image_path)
gray_multi_image = cv2.cvtColor(multi_image, cv2.COLOR_BGR2GRAY)

# Aplicar template matching
result_multi = cv2.matchTemplate(gray_multi_image, template, cv2.TM_CCOEFF_NORMED)
threshold_multi = 0.8
locations_multi = np.where(result_multi >= threshold_multi)

# Dibujar bounding boxes para cada coincidencia
for pt in zip(*locations_multi[::-1]):
    cv2.rectangle(multi_image, pt, (pt[0] + template_w, pt[1] + template_h), (0, 255, 0), 2)
    confidence = result_multi[pt[1], pt[0]]
    cv2.putText(multi_image, f'{confidence:.2f}', (pt[0], pt[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

# Mostrar el resultado
plt.imshow(cv2.cvtColor(multi_image, cv2.COLOR_BGR2RGB))
plt.title('Múltiples detecciones en coca_multi.png')
plt.axis('off')
plt.show()


## Parte 3: Generalización del algoritmo para todas las imágenes.
Vamos a generalizar el algoritmo para todas las imágenes de la carpeta y usaremos el método de supresión de no máximos (NMS) para evitar solapamientos innecesarios en las detecciones.

Usamos glob para iterar sobre todas las imágenes en la carpeta 'images'.

Implementamos la función non_max_suppression() para evitar múltiples detecciones del mismo logo muy cercanas entre sí.

Dibujamos los bounding boxes solo para las detecciones que quedan después de aplicar NMS.

In [ ]:
import glob

# Función para aplicar NMS
def non_max_suppression(boxes, scores, threshold=0.5):
    if len(boxes) == 0:
        return []

    boxes = np.array(boxes)
    scores = np.array(scores)

    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)

        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        overlap = (w * h) / areas[order[1:]]

        order = order[np.where(overlap <= threshold)[0] + 1]

    return keep

# Generalizar para todas las imágenes
all_images_paths = glob.glob('images/*.png')

for path in all_images_paths:
    # Leer la imagen
    image = cv2.imread(path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Aplicar template matching
    result = cv2.matchTemplate(gray_image, template, cv2.TM_CCOEFF_NORMED)
    threshold = 0.8
    locations = np.where(result >= threshold)

    # Guardar las coordenadas de las detecciones y sus puntuaciones
    boxes = []
    scores = []

    for pt in zip(*locations[::-1]):
        x1, y1 = pt
        x2, y2 = x1 + template_w, y1 + template_h
        confidence = result[pt[1], pt[0]]
        boxes.append([x1, y1, x2, y2])
        scores.append(confidence)

    # Aplicar NMS para eliminar solapamientos
    keep_indices = non_max_suppression(boxes, scores, threshold=0.5)

    # Dibujar los bounding boxes finales
    for idx in keep_indices:
        x1, y1, x2, y2 = boxes[idx]
        confidence = scores[idx]
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, f'{confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # Mostrar el resultado
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(f'Detección generalizada en {path}')
    plt.axis('off')
    plt.show()


## Parte 4: Puntos extra. Aplicar unsharp masking para expandir la zona de enfoque y devolver.
Vamos a aplicar unsharp masking para mejorar la nitidez de las imágenes y expandir la zona de enfoque. Esto resalta los detalles al aumentar el contraste de los bordes haciendo así que el logotipo sea más visible antes de la detección.

In [ ]:
def unsharp_mask(image, kernel_size=(5, 5), sigma=1.0, amount=1.5, threshold=0):
    # Aplicar suavizado Gaussian a la imagen
    blurred = cv2.GaussianBlur(image, kernel_size, sigma)

    # Calcular la máscara
    mask = cv2.addWeighted(image, 1 + amount, blurred, -amount, 0)

    # Aplicar umbral para mejorar detalles
    if threshold > 0:
        low_contrast_mask = np.abs(image - blurred) < threshold
        np.copyto(mask, image, where=low_contrast_mask)

    return mask

# Lista de imágenes a procesar
all_images_paths = glob.glob('images/*.png')

# Cargar el template
template = cv2.imread('template.png', cv2.IMREAD_GRAYSCALE)
template_h, template_w = template.shape

for path in all_images_paths:
    # Leer la imagen
    image = cv2.imread(path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Aplicar Unsharp Masking
    sharpened_image = unsharp_mask(gray_image, kernel_size=(5, 5), sigma=1.0, amount=1.5, threshold=10)

    # Aplicar template matching sobre la imagen mejorada
    result = cv2.matchTemplate(sharpened_image, template, cv2.TM_CCOEFF_NORMED)
    threshold = 0.8
    locations = np.where(result >= threshold)

    # Dibujar los bounding boxes para cada coincidencia
    for pt in zip(*locations[::-1]):
        cv2.rectangle(image, pt, (pt[0] + template_w, pt[1] + template_h), (0, 255, 0), 2)
        confidence = result[pt[1], pt[0]]
        cv2.putText(image, f'{confidence:.2f}', (pt[0], pt[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # Mostrar el resultado
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(f'Detección con Unsharp Masking en {path}')
    plt.axis('off')
    plt.show()


Lo que hicimos fue aplicar unsharp_mask a la imagen en escala de grises para resaltar los bordes y después realizamos template matching con la imagen mejorada para detectar el logotipo.